## Vehicle Traffic Analysis for Smart Street Lighting
> Here we perform Data visualisation of the acquired vehicle count data and determine optimal hours to keep the Street lights Dimmed.

In [3]:
# Author: Dilip Rajkumar
# Email: d.rajkumar@hbksaar.de
# License: MIT License

In [4]:
import pandas as pd
import numpy as np
from datetime import date
import yaml
import warnings
warnings.simplefilter('ignore') 

In [5]:
from bokeh.plotting import figure
from bokeh.layouts import row,column,layout,widgetbox
from bokeh.io import output_notebook,show,output_file
from bokeh.models import WMTSTileSource,ColumnDataSource,CategoricalColorMapper,HoverTool,Label
from bokeh.models.widgets import Slider,DateSlider,DatePicker
output_notebook() 

Loading BokehJS ...

In [6]:
# Reading the cleaned Hourly Vehicle Counts DataFrame
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTCG1elMKQ8RjGwfIhs8_F7uGbx5qvxEidnm0gXhG2-q-zrhPTzno5TyIILLDIWNe-fKLHtsMfrmzb5/pub?gid=1430444734&single=true&output=csv'
df_hourly = pd.read_csv(url,parse_dates = ['TimeStamps'],infer_datetime_format = True,index_col=['TimeStamps'])
df_hourly['Hour_of_Day'] = df_hourly.index.hour ## Add Hour of Day as Numerical Index
df_hourly.iloc[15:].head(6)

,VehicleCountperHour,DayorNight,Hour_of_Day
TimeStamps,,,
2018-02-23 06:01:01,32,Night,6
2018-02-23 07:01:08,72,Night,7
2018-02-23 08:01:24,57,Day,8
2018-02-23 09:01:39,32,Day,9
2018-02-23 10:01:55,68,Day,10
2018-02-23 11:02:11,81,Day,11


In [7]:
# Limit DataFrame to one Particular Date to initialize the plots
hour_value = 10
date_value = date(2018, 2, 23)
date_value = date_value.strftime('%Y-%m-%d')

### GeoMap Visualization of Vehicle Traffic Data

In [8]:
# Use this link to convert Lat Long to Web Mercarator co-ordinates 
# https://epsg.io/map#srs=3857&x=784328&y=6321000&z=18
city = x_range,y_range = ((781303,781373),(6316450,6316499))
urlmap = 'http://a.basemaps.cartocdn.com/light_all/{Z}/{X}/{Y}.png'
# urlmap = 'http://a.basemaps.cartocdn.com/dark_all/{Z}/{X}/{Y}.png'
attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"

In [9]:
def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df_city = pd.DataFrame(dict(name=["Waldhausweg, Saarbruecken"], lat=[49.2447058], lon=[7.0188925]))
wgs84_to_web_mercator(df_city)

,lat,lon,name,x,y
0,49.244706,7.018892,"Waldhausweg, Saarbruecken",781339.539033,6.316485e+06


In [10]:
def mapviz(doc):  
    p1 = figure(plot_width=425, plot_height=425,tools="",toolbar_location="above", x_range=x_range,y_range=y_range,
                title = "Hourly Vehicle Count at {}".format(df_city.name.iloc[0]),sizing_mode='scale_width')
    p1.axis.visible = False
    p1.add_tile(WMTSTileSource(url=urlmap, attribution=attribution))
    p1.circle(x=df_city['x'], y=df_city['y'], fill_color='white',line_color="red",line_width=4,size=42)
    annotation1 = Label(x=207, y=247, x_units='screen', y_units='screen',render_mode='css',background_fill_color='white',
             text='{}'.format(int(df_hourly.VehicleCountperHour.loc[date_value][df_hourly.Hour_of_Day == hour_value])),
            background_fill_alpha=0.0,text_color = 'black',text_font = 'arial',text_font_size = '12pt')
    p1.add_layout(annotation1)
    p2 = figure(plot_width=425, plot_height=425,tools='save',toolbar_location="above", x_range=x_range,y_range=y_range,
                title = "Daily Total Vehicle Count for {}".format(df_city.name.iloc[0]),sizing_mode='scale_width')
    p2.axis.visible = False
    p2.add_tile(WMTSTileSource(url=urlmap, attribution=attribution))
    p2.circle(x=df_city['x'], y=df_city['y'], fill_color='white',line_color="red",line_width=4,size=42)
    annotation2 = Label(x=200, y=247, x_units='screen', y_units='screen',text='{}'.format(df_hourly.loc[date_value].VehicleCountperHour.sum()),
                        render_mode='css',background_fill_color='white',background_fill_alpha=0.0,text_color = 'black',text_font = 'arial',text_font_size = '12pt')
    p2.add_layout(annotation2)
          
    def callback(attr,old,new):
        date_value = date_picker.value.strftime('%Y-%m-%d')
        hour_value = hour_slider.value
        annotation1.text = '{}'.format(int(df_hourly.VehicleCountperHour.loc[date_value][df_hourly.Hour_of_Day == hour_value]))
        annotation2.text = '{}'.format(df_hourly.loc[date_value].VehicleCountperHour.sum())             
   
    date_picker = DatePicker(min_date=date(2018,2,23), max_date=date(2018,2,24), value=date(2018,2,23), title="Select Date:")
    date_picker.on_change('value', callback)    
    hour_slider = Slider(title="Hour of Day", start=0, end=23,value=10, step=1)
    hour_slider.on_change('value', callback)
    doc.add_root(column(widgetbox(date_picker,hour_slider,sizing_mode ="scale_width"),row(p1,p2)))    

Now we can display our application using `show`, which will automatically create an `Application` that calls the plotting function (Eg: `mapviz`) using `FunctionHandler`. The end result is that the Bokeh server will call `mapviz` to build new documents for every new session that is opened.

In [11]:
show(mapviz)

### Vehicle Traffic Analysis and Classification

In [12]:
df_night = df_hourly[df_hourly.DayorNight == 'Night']
df_night = df_night[(df_night.index >= '22-2-2018') & (df_night.index < '25-2-2018')]
dates_unique = df_night.index.map(lambda t: t.date()).unique()
dates_unique = [dates_unique[i].strftime('%Y-%m-%d') for i in range(len(dates_unique))]
dates_unique

['2018-02-22', '2018-02-23', '2018-02-24']

#### Classifying Street Light Operation Hours
  We determine the best time to keep a street light *DIMMED* using the [interquartile range](https://en.wikipedia.org/wiki/Interquartile_range) of the vehicle traffic distribution. In the codeblock below, we classify all hours of the **night** where the night time vehicle traffic is less than the third quartile (Q3 or *75th percentile*) as optimal hours to keep a street light dimmed. This ensures that street lights are turned on at *FULL* Brightness during peak hours and dimmed during Off-Peak Hours.

In [13]:
statelist = []
countlist = []
Timestamps = []
for e in dates_unique:    
    df_night_date = df_night[e].reset_index()
    IQR75 = int(np.percentile(df_night_date.VehicleCountperHour, 75))
    df_night_date['StreetLightState'] = 'FULL'
    dim_index = df_night_date[df_night_date.VehicleCountperHour < IQR75].index
    df_night_date['StreetLightState'].iloc[dim_index] = 'DIM'
    list1 = df_night_date.StreetLightState.tolist()
    statelist.extend(list1)
    list2 = df_night_date.VehicleCountperHour.tolist()
    countlist.extend(list2)
    list3 = df_night_date.TimeStamps.tolist()
    Timestamps.extend(list3)

In [14]:
df_night_lightstate = pd.DataFrame({'Timestamps': Timestamps, 'VehicleCountperHour': countlist, 'StreetLightState':statelist})
df_night_lightstate = df_night_lightstate[['Timestamps','VehicleCountperHour','StreetLightState']]
df_night_lightstate = df_night_lightstate.set_index('Timestamps')
df_night_lightstate.head()

,VehicleCountperHour,StreetLightState
Timestamps,,
2018-02-22 00:34:06,7,DIM
2018-02-22 01:34:22,12,DIM
2018-02-22 02:34:38,7,DIM
2018-02-22 03:34:52,4,DIM
2018-02-22 04:35:09,14,DIM


In [15]:
## Custom function to Set the bar for each hour at a decent width
## https://stackoverflow.com/questions/40185561/auto-set-vbar-line-width-based-on-x-range-in-bokeh
def get_width(df):
    '''Pass DataFrame as input attribute to the function'''
    mindate = min(df.index)
    maxdate = max(df.index)
    return 0.75 * (maxdate-mindate).total_seconds()*1000 / len(df.index)

In [16]:
def TrafficPlot(doc):
    source1 = ColumnDataSource(data=dict(x=df_hourly.loc[date_value].index, 
                                         y=df_hourly.loc[date_value].VehicleCountperHour,
                                         label = df_hourly.loc[date_value].DayorNight.tolist()))
    source2 = ColumnDataSource(data=dict(a = df_night_lightstate.loc[date_value].index,
                                     b = df_night_lightstate.loc[date_value].VehicleCountperHour,
                                     label1 = df_night_lightstate.loc[date_value].StreetLightState.tolist()))
    hover = HoverTool(names=['Traffic'],tooltips=[("Timestamp", "@x{%F %T}"),("CountValue", "@y")],formatters={"x": "datetime"})
    hover.point_policy= "snap_to_data"
    color_mapper1 = CategoricalColorMapper(factors=['Day', 'Night'], palette=['#ffe47c','#3692b6'])
    color_mapper2 = CategoricalColorMapper(factors=['FULL', 'DIM'], palette=['#ea2539','#48ad1d'])    
    p = figure(title = "Hourly Vehicle Count",width = 960, height=540,sizing_mode='scale_width',
              x_axis_type='datetime',y_range=(0, (df_hourly.VehicleCountperHour.max()+1)))
    p.line('x','y',color="Red",alpha = 0.55, source = source1)
    p.vbar('x', top='y',width=get_width(df_hourly),alpha = 0.75,source=source1,name='Traffic',color={'field': 'label', 'transform': color_mapper1},legend='label')
    p.add_tools(hover)
    p.circle('a','b',size=15,alpha = 0.5,source=source2,color={'field': 'label1', 'transform': color_mapper2},legend='label1')
    annotation1 = Label(x=20, y=460, x_units='screen', y_units='screen',render_mode='css',background_fill_color='white',
                        text='Busiest Hour of the Day: {} cars @ {}'.format(df_hourly.loc[date_value].VehicleCountperHour.max(),df_hourly.loc[date_value].VehicleCountperHour.idxmax()),
                         background_fill_alpha=0.45,text_font = 'arial',text_font_size = '9pt', text_color='#828282')
    annotation2 = Label(x=20, y=440, x_units='screen', y_units='screen', render_mode='css',background_fill_color='white',
                        text='Average Hourly Vehicle Traffic: ~ {} per hour'.format(round(df_hourly.loc[date_value].VehicleCountperHour.mean())),background_fill_alpha=0.55,text_font = 'arial',text_font_size = '9pt', text_color='#828282')
    annotation3 = Label(x=20, y=-40, x_units='screen', y_units='screen',render_mode='css',background_fill_color='white',
                        text='Total Car movement in the day:{}'.format(df_hourly.loc[date_value].VehicleCountperHour.sum()),background_fill_alpha=0.55,text_font = 'times',text_font_size = '10pt',text_font_style='bold')   
    p.add_layout(annotation1)
    p.add_layout(annotation2)
    p.add_layout(annotation3)
    
    def callback(attr,old,new):
        date_value = new.strftime('%Y-%m-%d')
        new_data = dict(x=df_hourly.loc[date_value].index,
            y=df_hourly.loc[date_value].VehicleCountperHour,
            label = df_hourly.loc[date_value].DayorNight.tolist())
        source1.data = new_data
        new_data2 = dict(a = df_night_lightstate.loc[date_value].index,
                         b = df_night_lightstate.loc[date_value].VehicleCountperHour,
                         label1 = df_night_lightstate.loc[date_value].StreetLightState.tolist())
        source2.data = new_data2
        annotation1.text = 'Busiest Hour of the Day: {} cars @ {}'.format(df_hourly.loc[date_value].VehicleCountperHour.max(),df_hourly.loc[date_value].VehicleCountperHour.idxmax())
        annotation2.text = 'Average Hourly Vehicle Traffic: ~ {} per hour'.format(round(df_hourly.loc[date_value].VehicleCountperHour.mean()))
        annotation3.text = 'Total Car movement in the day: {}'.format((df_hourly.loc[date_value].VehicleCountperHour.sum()))
    
    slider = DateSlider(start=date(2018,2,22), end=date(2018,2,24), value=date(2018,2,23), step=1, title="Date")
    slider.on_change('value', callback)
    doc.add_root(column(slider,p))   

The plot below shows the vehicle traffic distribution for the selected date. The particular hours where street lights can be dimmed are marked in <font color='#48ad1d'>GREEN</font> color and the hours where street lights can be operated at FULL Brightness is marked in <font color='#ea2539'>RED</font> color.

In [17]:
show(TrafficPlot, notebook_handle = True)

### Energy Savings
  Since we have only two days of continous uninterrupted vehicle count data we will subset our data frame to these two dates 23/02/2018 (Friday) and 24/02/2018 (Saturday). In these two dates,there is 14 hours of Night time when the street lights will be functioning. We use the available vehicle count data and generalize it to any arbitrary street to quantify the energy savings for this weekday (Friday) and a weekend (Saturday). 

In [18]:
# Subset dataframe to just two days
df_night = df_hourly[df_hourly.DayorNight == 'Night']
df_night = df_night[(df_night.index >= '23-2-2018') & (df_night.index < '25-2-2018')]
dates_unique = df_night.index.map(lambda t: t.date()).unique()
dates_unique = [dates_unique[i].strftime('%Y-%m-%d') for i in range(len(dates_unique))]

In [19]:
# Restructuring data into a new data frame
dimhours = []
fullbrighthours = []
totalnighthours = []
for e in dates_unique:
    list1 = int(df_night[e].DayorNight.count())
    totalnighthours.append(list1)
    list2 = int(df_night_lightstate.StreetLightState[e][df_night_lightstate.StreetLightState == 'DIM'].count())
    dimhours.append(list2)
    list3 = int(df_night_lightstate.StreetLightState[e][df_night_lightstate.StreetLightState == 'FULL'].count())
    fullbrighthours.append(list3)

In [20]:
dates_list = df_night.index.map(lambda t: t.date()).unique()
df_dimhrs = pd.DataFrame({'Date': dates_list, 'NightHours': totalnighthours, 'LightDimHours':dimhours, 'FullBrightHours': fullbrighthours})
df_dimhrs = df_dimhrs[['Date','NightHours','FullBrightHours','LightDimHours']]
df_dimhrs = df_dimhrs.set_index('Date')
df_dimhrs

,NightHours,FullBrightHours,LightDimHours
Date,,,
2018-02-23,14,4,10
2018-02-24,14,5,9


In [21]:
dates_unique = [dates_list[i].strftime('%a, %d-%m-%Y') for i in range(len(dates_list))]
print(dates_unique)

['Fri, 23-02-2018', 'Sat, 24-02-2018']


### Formula for Calculating Street Light Dimming Hours

In [22]:
# Initialize Values to start the plot
lights = 10
power = 50
dimming_factor = 0.25

The formula for calculating Energy Consumption is as follows:<br>
1. In a normal scenario, when the street lights are switched ON at *FULL* Brightness during the entire night, we calculate:

        Energy Consumed = (No. of Hours of Operation ) x (Power Rating of each light) x (No.of lights in the street)
2. In the energy savings scenario where street lights are *DIMMED* when vehicle traffic is less and increased to *FULL* Brightness during peak vehicle traffic hours, we calculate:

        Energy Consumed = { (No. of Hours of Operation at FULL Brightness ) x (Power Rating of each light ) x (No.of lights in the street ) } +  { (No. of Hours of Operation at DIMMED Brightness ) x (Power Rating of each light ) x (Dimming Factor ) x (No.of lights in the street ) }

**Dimming Factor**:
    Dimming Factor is the percentage of full brightness you want to dim the lights. For eg: Dimming Factor = 0.25 means 25% of FULL Brightness . For the calculations here it is assumed that the dimming the street light reduces it's luminosity and power consumption proportionately.

 **Limitation**:
The Energy Savings scheme proposed here assumes that street lights can be dimmed. However some older street light technologies <sup>[1]</sup> like *High Pressure Sodium Vapor* cannot be dimmed instantly with proportionate decrease in luminosity and power rating, compared to modern technologies like LEDs.

In [23]:
def energyplot(doc):
    source1 = ColumnDataSource(data = dict(normal=['Normal'],max_energy = [(14*lights*power*0.001)]))
    source2 = ColumnDataSource(data=dict(dates = dates_unique,energy=[(a*(lights*power*0.001)) + (b*(lights*power*dimming_factor*0.001)) for a, b in zip(df_dimhrs.FullBrightHours.tolist(),df_dimhrs.LightDimHours.tolist())]))
    hover = HoverTool(names=['DimDays'],tooltips=[("Energy Consumption", "@energy kWh")])
    hover_fullday = HoverTool(names=['FullDay'],tooltips=[("Energy Consumption", "@max_energy kWh")])
    f = figure(plot_width=960,plot_height=540,y_range=['Normal']+dates_unique,toolbar_location=None,tools=[hover], 
               title="Energy Consumption in a Street after Dimming Street Lights",sizing_mode='scale_width')
    f.hbar(y='normal',left=0, right = 'max_energy',source=source1, height=0.25,color="#ea2539",alpha = 0.65,legend="FULL",name='FullDay')
    f.hbar(y='dates',left=0, right='energy', height=0.25, source=source2,color = "#48ad1d",alpha = 0.65,legend="DIM",name='DimDays')
    f.ygrid.grid_line_color = None
    f.x_range.start = 0
    f.x_range.end = 61 ##Set this to a value to fix the scale of the X-Axis
    f.xaxis.axis_label = "kW-Hr"
    f.add_tools(hover_fullday)      
          
    def callback(attrname,old,new):        
        lights = Lights_Slider.value
        power = PowerSlider.value
        dimming_factor = DimmingSlider.value
        source1.data = dict(normal=['Normal'],max_energy = [(14*lights*power*0.001)])        
        source2.data = dict(dates = dates_unique,energy=[(a*(lights*power*0.001)) + (b*(lights*power*dimming_factor*0.001)) for a, b in zip(df_dimhrs.FullBrightHours.tolist(),df_dimhrs.LightDimHours.tolist())])
        
    Lights_Slider = Slider(start=0, end=20, value=10, step=1, title="No. of Street Lights")
    PowerSlider = Slider(start=10, end=200, value=50, step=1, title="Power (Watts) of each Light")
    DimmingSlider = Slider(start=0.25, end=1, value=0.25, step=.25, title="Dimming Factor")
    for w in [Lights_Slider, PowerSlider, DimmingSlider]:
        w.on_change('value', callback)
    doc.add_root(column(widgetbox(Lights_Slider,PowerSlider,DimmingSlider),f)) 

In [24]:
show(energyplot)

### Concluding Thoughts:
* Only two days of vehicle count data are used to present the visualizations, it would be ideal to have data for a longer duration like a month or ideally a year so we can assess vehicle traffic metrics on average for each day of the week for an entire year.

#### Economic metrics about energy savings:
 * We can see that for a street with 10 nos of 50 Watt street lights and a dimming factor of 0.25 we get an energy savings of 3.45 kWh per day on average, which translates to a yearly energy savings of ~ 1260 kWh.
 * Taking the unit cost of Electricity in Saarbrücken<sup>[3]</sup> to be ~0.07 € per kWh, this amounts to about 88 € per year. With several thousand street lights in Saarbrücken, this amount could easily exceed 1 million €

#### References:
1. https://electronics.stackexchange.com/questions/24978/how-do-you-dim-a-streetlight-bulb 
2. https://bokeh.pydata.org/en/latest/docs/user_guide/categorical.html
3. https://www.energie-saarlorlux.com/geschaeftskunden/fernwaerme/preise/